In [1]:
import tensorflow as tf


C:\Users\User\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\envs\tensorflow-gpu\lib\s

In [2]:
tf.__version__

'1.14.0'

In [3]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation,MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
from keras.layers import Input

from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from keras.optimizers import *
from keras.models import Model,Sequential
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
import numpy as np
import pandas as pd 
from numpy import zeros, newaxis
import cv2 
import matplotlib.pyplot as plt


# khaled edits
from imgaug import augmenters as iaa

Using TensorFlow backend.


In [4]:
metaData = pd.read_csv("./Chest_xray_Corona_Metadata.csv")
metaData

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...,...
5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [5]:
normalMetaData=metaData.loc[metaData['Label'] == 'Normal']
normalMetaData

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...,...
5515,5538,NORMAL2-IM-0341-0001.jpeg,Normal,TEST,NaN,NaN
5516,5539,NORMAL2-IM-0340-0001.jpeg,Normal,TEST,NaN,NaN
5517,5540,NORMAL2-IM-0339-0001.jpeg,Normal,TEST,NaN,NaN
5518,5541,NORMAL2-IM-0338-0001.jpeg,Normal,TEST,NaN,NaN


In [6]:
PnemoniaMetaData=metaData.loc[metaData['Label'] == 'Pnemonia']
PnemoniaMetaData

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
1341,1341,person101_virus_188.jpeg,Pnemonia,TRAIN,NaN,Virus
1342,1342,person101_virus_187.jpeg,Pnemonia,TRAIN,NaN,Virus
1343,1343,person1019_virus_1708.jpeg,Pnemonia,TRAIN,NaN,Virus
1344,1344,person1019_virus_1707.jpeg,Pnemonia,TRAIN,NaN,Virus
1345,1345,person1019_bacteria_2950.jpeg,Pnemonia,TRAIN,NaN,bacteria
...,...,...,...,...,...,...
5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [7]:
VirusPnemoniaMetaData=PnemoniaMetaData.loc[PnemoniaMetaData['Label_1_Virus_category'] == 'Virus']
VirusPnemoniaMetaData

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
1341,1341,person101_virus_188.jpeg,Pnemonia,TRAIN,NaN,Virus
1342,1342,person101_virus_187.jpeg,Pnemonia,TRAIN,NaN,Virus
1343,1343,person1019_virus_1708.jpeg,Pnemonia,TRAIN,NaN,Virus
1344,1344,person1019_virus_1707.jpeg,Pnemonia,TRAIN,NaN,Virus
1346,1346,person1018_virus_1706.jpeg,Pnemonia,TRAIN,NaN,Virus
...,...,...,...,...,...,...
5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [8]:
BacteriaPnemoniaMetaData=PnemoniaMetaData.loc[PnemoniaMetaData['Label_1_Virus_category'] == 'bacteria']
BacteriaPnemoniaMetaData

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
1345,1345,person1019_bacteria_2950.jpeg,Pnemonia,TRAIN,NaN,bacteria
1347,1347,person1018_bacteria_2949.jpeg,Pnemonia,TRAIN,NaN,bacteria
1348,1348,person1017_bacteria_2948.jpeg,Pnemonia,TRAIN,NaN,bacteria
1350,1350,person1016_bacteria_2947.jpeg,Pnemonia,TRAIN,NaN,bacteria
1353,1353,person1014_bacteria_2945.jpeg,Pnemonia,TRAIN,NaN,bacteria
...,...,...,...,...,...,...
5859,5882,person173_bacteria_830.jpeg,Pnemonia,TEST,NaN,bacteria
5860,5883,person173_bacteria_829.jpeg,Pnemonia,TEST,NaN,bacteria
5861,5884,person172_bacteria_828.jpeg,Pnemonia,TEST,NaN,bacteria
5862,5885,person172_bacteria_827.jpeg,Pnemonia,TEST,NaN,bacteria


In [9]:
normalMetaDataTrain=normalMetaData.loc[normalMetaData['Dataset_type'] == 'TRAIN']
normalMetaDataTrain

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...,...
1337,1337,NORMAL2-IM-1334-0001.jpeg,Normal,TRAIN,NaN,NaN
1338,1338,NORMAL2-IM-1333-0001.jpeg,Normal,TRAIN,NaN,NaN
1339,1339,NORMAL2-IM-1332-0001.jpeg,Normal,TRAIN,NaN,NaN
1340,1340,NORMAL2-IM-1330-0001.jpeg,Normal,TRAIN,NaN,NaN


In [10]:
normalMetaDataTest=normalMetaData.loc[normalMetaData['Dataset_type'] == 'TEST']
normalMetaDataTest

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5286,5309,IM-0021-0001.jpeg,Normal,TEST,NaN,NaN
5287,5310,IM-0019-0001.jpeg,Normal,TEST,NaN,NaN
5288,5311,IM-0017-0001.jpeg,Normal,TEST,NaN,NaN
5289,5312,IM-0016-0001.jpeg,Normal,TEST,NaN,NaN
5290,5313,IM-0015-0001.jpeg,Normal,TEST,NaN,NaN
...,...,...,...,...,...,...
5515,5538,NORMAL2-IM-0341-0001.jpeg,Normal,TEST,NaN,NaN
5516,5539,NORMAL2-IM-0340-0001.jpeg,Normal,TEST,NaN,NaN
5517,5540,NORMAL2-IM-0339-0001.jpeg,Normal,TEST,NaN,NaN
5518,5541,NORMAL2-IM-0338-0001.jpeg,Normal,TEST,NaN,NaN


In [11]:
PnemoniaMetaDataTrain=PnemoniaMetaData.loc[PnemoniaMetaData['Dataset_type'] == 'TRAIN']
PnemoniaMetaDataTrain

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
1341,1341,person101_virus_188.jpeg,Pnemonia,TRAIN,NaN,Virus
1342,1342,person101_virus_187.jpeg,Pnemonia,TRAIN,NaN,Virus
1343,1343,person1019_virus_1708.jpeg,Pnemonia,TRAIN,NaN,Virus
1344,1344,person1019_virus_1707.jpeg,Pnemonia,TRAIN,NaN,Virus
1345,1345,person1019_bacteria_2950.jpeg,Pnemonia,TRAIN,NaN,bacteria
...,...,...,...,...,...,...
5281,5304,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,Pnemonia,TRAIN,COVID-19,Virus
5282,5305,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,Pnemonia,TRAIN,COVID-19,Virus
5283,5306,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,Pnemonia,TRAIN,COVID-19,Virus
5284,5307,1-s2.0-S0140673620303706-fx1_lrg.jpg,Pnemonia,TRAIN,COVID-19,Virus


In [12]:
PnemoniaMetaDataTest=PnemoniaMetaData.loc[PnemoniaMetaData['Dataset_type'] == 'TEST']

PnemoniaMetaDataTest

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5520,5543,person1631_virus_2826.jpeg,Pnemonia,TEST,NaN,Virus
5521,5544,person1629_virus_2823.jpeg,Pnemonia,TEST,NaN,Virus
5522,5545,person1628_virus_2822.jpeg,Pnemonia,TEST,NaN,Virus
5523,5546,person1628_virus_2821.jpeg,Pnemonia,TEST,NaN,Virus
5524,5547,person1627_virus_2819.jpeg,Pnemonia,TEST,NaN,Virus
...,...,...,...,...,...,...
5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [13]:
def zoom(image):
  zoom = iaa.Affine(scale=(1, 1.5))          # this scale of zoom in image ratio of 30%
  image = zoom.augment_image(image)
  return image

def pan(image):                              # image shifted over x-y axis helps to highlights region 
  pan = iaa.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})        # 10% to left or right and same ratio to up-down
  image = pan.augment_image(image)
  return image

def brightness(image):                       # change brightness by multiplying pixels as mask
    brightness = iaa.Multiply((0.7, 1.7))    # 0.2-1.2 is limits of multipler 
    image = brightness.augment_image(image)  # note : if value is less than 1, means darker image
    return image

def flip(image):                             # flipping only about x_axis
    image = cv2.flip(image,1)               
    return image

def random_augment(image):
    if np.random.rand() < 0.1:
        image = pan(image)
    if np.random.rand() < 0.6:
        image = zoom(image)
    if np.random.rand() < 0.6:
        image = brightness(image)
  # if np.random.rand() < 0.1:
    #   image = flip(image)
    
    return image

In [14]:
X_Test_level1 = []
Y_Test_level1= []

X_val_level1 = []
Y_val_level1= []

for i in range (0,390):
    path='./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'+ str(PnemoniaMetaDataTest.iloc[i]['X_ray_image_name'])
    reshapedimage =cv2.resize(cv2.imread(path, 1), (224,224))
    if i %2 :
        X_Test_level1.append(reshapedimage)
        Y_Test_level1.append(1.0)
    else :
        X_val_level1.append(reshapedimage)
        Y_val_level1.append(1.0)
for i in range (0,234):
    path='./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'+ str(normalMetaDataTest.iloc[i]['X_ray_image_name'])
    reshapedimage =cv2.resize(cv2.imread(path, 1), (224,224))
    if i %2 :
        X_Test_level1.append(reshapedimage)
        Y_Test_level1.append(0.0)
    else :
        X_val_level1.append(reshapedimage)
        Y_val_level1.append(0.0)


X_Test_level1=np.array(X_Test_level1)
Y_Test_level1=np.array(Y_Test_level1)

X_val_level1=np.array(X_val_level1)
Y_val_level1=np.array(Y_val_level1)

X_Train_level1 = []
Y_Train_level1= []
for i in range (0,1500):
    path='./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'+ str(PnemoniaMetaDataTrain.iloc[i]['X_ray_image_name'])
    reshapedimage =cv2.resize(cv2.imread(path, 1), (224,224))
    if i% 1 == 0:
        aug_image = random_augment(reshapedimage)
        X_Train_level1.append(aug_image)
        Y_Train_level1.append(1.0)
    X_Train_level1.append(reshapedimage)
    Y_Train_level1.append(1.0)
for i in range (0,1342):
    path='./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'+ str(normalMetaDataTrain.iloc[i]['X_ray_image_name'])
    reshapedimage =cv2.resize(cv2.imread(path, 1), (224,224))
    if i% 1 == 0:
        aug_image = random_augment(reshapedimage)
        X_Train_level1.append(aug_image)
        Y_Train_level1.append(0.0)
    X_Train_level1.append(reshapedimage)
    Y_Train_level1.append(0.0)
    


X_Train_level1=np.array(X_Train_level1)
Y_Train_level1=np.array(Y_Train_level1)

In [17]:
print(X_Train_level1.shape)
print(X_Test_level1.shape)
print(X_val_level1.shape)

(5684, 224, 224, 3)
(312, 224, 224, 3)
(312, 224, 224, 3)


In [15]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_Train_level1),
                                                 Y_Train_level1)


In [16]:
InputShape=(224,224,3)
act='relu'
def VGG():
    inputs = Input(shape=InputShape)

# First conv block
    x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
    x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
    x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
    x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.5)(x)

# Fifth conv block
    x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.5)(x)

# FC layer
    x = Flatten()(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(rate=0.7)(x)
    x = Dense(units=128, activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=64, activation='relu')(x)
    x = Dropout(rate=0.3)(x)

# Output layer
    output = Dense(units=1, activation='sigmoid')(x)

# Creating model and compiling
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',metrics=["binary_accuracy"])
    checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
    lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

    return model

In [23]:
modelVGG1=VGG()

hist=modelVGG1.fit(X_Train_level1, Y_Train_level1, epochs=50, batch_size=64,
                   class_weight= class_weights,
                   validation_data=(X_val_level1, Y_val_level1),
                   shuffle = 1)
print(modelVGG1.evaluate(X_Test_level1, Y_Test_level1, verbose=1))

Train on 5684 samples, validate on 312 samples
Epoch 1/50
5684/5684 [==============================] - 24s 4ms/step - loss: 0.3442 - binary_accuracy: 0.8614 - val_loss: 0.7048 - val_binary_accuracy: 0.6250
Epoch 2/50
5684/5684 [==============================] - 20s 4ms/step - loss: 0.2219 - binary_accuracy: 0.9238 - val_loss: 0.6652 - val_binary_accuracy: 0.6250
Epoch 3/50
1664/5684 [=======>......................] - ETA: 14s - loss: 0.2064 - binary_accuracy: 0.9243

KeyboardInterrupt: 

In [17]:
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, Input, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications import DenseNet121

In [34]:
densenet_model = DenseNet121(weights =None, include_top = False)
densenet_model.trainable = False


In [35]:
densenet_model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics= ['accuracy'])
